In [6]:
import numpy as np
import random
import math
from scipy.stats import chi2,  norm

## Ejercicio 1

In [16]:
def sim_mendel(Nsims=1000):
    pvalor = 0
    for _ in range(Nsims):
        N1 = np.random.binomial(564, 1/4)
        N2 = np.random.binomial(564-N1, (1/2)/(1-1/4))
        N3 = 564 - N2 - N1
        T = (N1 - 564*1/4)**2 / (564*1/4)
        T += (N2 - 564*1/2)**2 / (564*1/2)
        T += (N3 - 564*1/4)**2 / (564*1/4)
        if T > 0.8617:
            pvalor += 1
    return pvalor/Nsims
    
print(f"Valor estimado de p-valor: {sim_mendel()}")
print(f"Valor de p-valor con chi^2: {0.64996}")

Valor estimado de p-valor: 0.668
Valor de p-valor con chi^2: 0.64996


## Ejercicio 2

In [17]:
def sim_dados(Nsims=1000):
    pvalor = 0
    for _ in range(Nsims):
        muestra = np.random.randint(1, 7, 1000)
        N = [0 for i in range(7)]
        for i in muestra:
            N[i] += 1
        T = 0
        for i in range(1, 7):
            T += (N[i] - 1000*1/6)**2 / (1000*1/6)
        if T > 2.18:
            pvalor += 1
    return pvalor/Nsims
        
print(f"Valor estimado de p-valor: {sim_dados()}")
print(f"Valor de p-valor con chi^2: {0.82372}")

Valor estimado de p-valor: 0.814
Valor de p-valor con chi^2: 0.82372


## Ejercicio 3

In [8]:
def estadistico_d(muestra, f):
    muestra.sort()
    m = 0
    for i in range(1, len(muestra)+1):
        x = muestra[i-1]
        n = len(muestra)
        m = max(m, (i)/n - f(x), f(x) - (i-1)/n)
    return m

d = estadistico_d([0.06, 0.12, 0.18, 0.27, 0.33, 0.36, 0.72, 0.74, 0.77, 0.83], (lambda x: x))

print("Valor del estadístico D:", d)

Valor del estadístico D: 0.24


In [9]:
def pvalor_simulado(d_KS, n, Nsims=10000):
    pvalor = 0
    for _ in range(Nsims):
        uniformes = np.random.uniform(0,1,n)
        uniformes.sort()
        d_j= estadistico_d(uniformes, (lambda x: x))
        if d_j > d_KS:
            pvalor += 1
    return pvalor/Nsims

print("Valor estimado del p-valor:", pvalor_simulado(d, 10))

Valor estimado del p-valor: 0.533


## Ejercicio 4

In [34]:
d = estadistico_d([86.0, 133.0, 75.0, 22.0, 11.0, 144.0, 78.0, 122.0, 8.0, 146.0, 33.0, 41.0, 99.0], 
                (lambda x: 1 - math.exp(-x/50)))

print("Valor del estadístico D:", d)
print("Valor estimado del p-valor:", pvalor_simulado(d, 13))

Valor del estadístico D: 0.3922544552361856
Valor estimado del p-valor: 0.021


## Ejercicio 5

In [21]:
M = [6, 7, 3, 4, 7, 3, 7, 2, 6, 3, 7, 8, 2, 1, 3, 5, 8, 7]
media_muestral = sum(M)/len(M)
estimador_p = media_muestral/8

print("Media muestral:", media_muestral)
print("Estimador de p:", estimador_p)

N = [0 for i in range(max(M)+1)]
for i in M:
    N[i] += 1
for i in range(len(N)):
    print(f"Frecuencia de N{i} = {N[i]}")

Media muestral: 4.944444444444445
Estimador de p: 0.6180555555555556
Frecuencia de N0 = 0
Frecuencia de N1 = 1
Frecuencia de N2 = 2
Frecuencia de N3 = 4
Frecuencia de N4 = 1
Frecuencia de N5 = 1
Frecuencia de N6 = 2
Frecuencia de N7 = 5
Frecuencia de N8 = 2


In [22]:
def prob_binomial(n, p, x):
    c = math.factorial(n)/(math.factorial(x)*math.factorial(n-x))
    pp = p ** x
    np = (1-p)**(n-x)
    return (c * pp * np)
    
def T_estadistico_bin(N, n, p):
    T = 0
    for i in range(len(N)):
        den = sum(N)*prob_binomial(n, p, i)
        T += (N[i] - den)**2 / den
    return T

t0 = T_estadistico_bin(N, 8, estimador_p)
       
print("T estadístico ejercicio 5:", t0)
print("p-valor con chi^2:", 1 - chi2.cdf(t0, 7))

T estadístico ejercicio 5: 31.499330934155324
p-valor con chi^2: 5.0279943204278865e-05


In [23]:
def sim_bin(Nsims=1000):
    pvalor = 0
    for _ in range(Nsims):
        muestra = np.random.binomial(n=8, p=estimador_p, size=18)
        N = [0 for i in range(9)]
        m_muestral = sum(muestra)/len(muestra)
        p_estimado = m_muestral/8
        for i in muestra:
            N[i] += 1
        T_est = T_estadistico_bin(N, 8, p_estimado)
        if T_est > t0:
            pvalor += 1
    return pvalor/Nsims

print("Valor estimado del p-valor simulando frecuencias:", sim_bin())

Valor estimado del p-valor simulando frecuencias: 0.009


## Ejercicio 6

In [24]:
def t_ej6(N, P):
    T = 0
    for i in range(1, len(N)):
        T += (N[i] - sum(N)*P[i]) ** 2 / (sum(N)*P[i])
    return T

N = [0, 188, 138, 87, 65, 48, 32, 30, 34, 13, 2]
P = [0, 0.31, 0.22, 0.12, 0.10, 0.08, 0.06, 0.04, 0.04, 0.02, 0.01]
t0 = t_ej6(N, P)

print("Valor del estadístico T:", t0)
print("Valor de p-valor usando chi^2:", 1 - chi2.cdf(t0, 9))

Valor del estadístico T: 9.810370888711903
Valor de p-valor usando chi^2: 0.36605389988682624


In [25]:
def generar_tiro():
    F = [0.0, 0.31, 0.53, 0.65, 0.75, 0.83, 0.89, 0.93, 0.97, 0.99, 1]
    U = random.random()
    i = 0
    while U > F[i]:
        i += 1
    return i

def sim_rueda(P, t0, Nsims=1000):
    pvalor = 0
    for _ in range(Nsims):
        muestra = []
        frec = [0 for i in range(11)]
        for i in range(637):
            x = generar_tiro()
            muestra.append(x)
            frec[x] += 1
        T_est = 0
        for i in range(1, len(N)):
            T_est += (frec[i] - len(muestra)*P[i])**2 / (len(muestra)*P[i])
        if T_est > t0:
            pvalor += 1
    return pvalor/Nsims
    
print("Valor de p-valor estimado generando las frecuencias:", sim_rueda(P, t0))        

Valor de p-valor estimado generando las frecuencias: 0.372


## Ejercicio 7

In [31]:
def sim_ej7():
    print("H0: La muestra tiene distribución esponencial con media 1")
    muestra = np.random.exponential(scale=1, size=30)
    d = estadistico_d(muestra, (lambda x: 1 - math.exp(-x)))
    pvalor = pvalor_simulado(d, 30)
    if pvalor > 0.05:
        print(f"No rechazamos H0 ya que: {pvalor} > 0.05")
    else:
        print(f"Rechazamos H0 ya que: {pvalor} < 0.05")
        
sim_ej7()

H0: La muestra tiene distribución esponencial con media 1
No rechazamos H0 ya que: 0.4758 > 0.05


## Ejercicio 8

In [16]:
def normal_acum(x):
    if x >= 0:
        Fi = math.erf(x/math.sqrt(2.))/2.+0.5
    else:
        Fi = 0.5-math.erf(-x/math.sqrt(2.))/2.
    return Fi

def rt(df): # df grados de libertad
    x = random.gauss(0.0, 1.0)
    y = 2.0*random.gammavariate(0.5*df, 2.0)
    return x / (math.sqrt(y/df))

def normal_tstudents(n):
    muestra = []
    for _ in range(n):
        muestra.append(rt(11))
    d = estadistico_d(muestra, normal_acum)
    pvalor = pvalor_simulado(d, n)
    return d, pvalor

print("n = 10", normal_tstudents(10))
print("n = 20", normal_tstudents(20))
print("n = 100", normal_tstudents(100))
print("n = 1000", normal_tstudents(1000))

n = 10 (0.26552644016308957, 0.4028)
n = 20 (0.2337478637679043, 0.1884)
n = 100 (0.12495125778557822, 0.0814)
n = 1000 (0.09188062636346134, 0.0)


## Ejercicio 9

In [36]:
M = [1.6, 10.3, 3.5, 13.5, 18.4, 7.7, 24.3, 10.7, 8.4, 4.9, 7.9, 12, 16.2, 6.8, 14.7]
media_muestral = sum(M)/len(M)
estimador_lambda = 1/media_muestral
d0 = estadistico_d(M, (lambda x: 1 - math.exp(-x*estimador_lambda)))

print("Valor estadístico d:", d0)
print("Valor estimado de p-valor con uniformes:", pvalor_simulado(d0, len(M)))

def pvalor_exp(lamda, d0, size, Nsims=10000):
    pvalor = 0
    for i in range(Nsims):
        muestra = np.random.exponential(1/lamda, size)
        media_muestral = sum(muestra)/len(muestra)
        lambda_estimado = 1/media_muestral
        d_j = estadistico_d(muestra, (lambda x: 1 - math.exp(-x*lambda_estimado)))
        if d_j > d0:
            pvalor += 1
    return pvalor/Nsims
        
print("Valor estimado de p-valor con exponenciales", pvalor_exp(estimador_lambda, d0, len(M)))
print("En ambos casos p-valor > 0.01, con un nivel de rechazo del 1% no se rechaza H0")

Valor estadístico d: 0.26949936321059237
Valor estimado de p-valor con uniformes: 0.1877
Valor estimado de p-valor con exponenciales 0.0475
En ambos casos p-valor > 0.01, con un nivel de rechazo del 1% no se rechaza H0


## Ejercicio 10

In [46]:
M = [91.9, 97.8, 111.4, 122.3, 105.4, 95.0, 103.8, 99.6, 96.6, 119.3, 104.8, 101.7]
media_muestral = sum(M)/len(M)
var_m = 0
for i in M:
    var_m += (i - media_muestral)**2
desvio_m = math.sqrt(var_m/(len(M)-1))

mu_estimado = media_muestral
sigma_estimado = desvio_m
d0 = estadistico_d(M, (lambda x: norm.cdf(x, mu_estimado, sigma_estimado)))
print("Estadístico d:", d0)
print("Valor estimado de p-valor con uniformes:", pvalor_simulado(d0, len(M)))

def pvalor_normal(mu, sigma, d0, size, Nsims=1000):
    pvalor = 0
    for i in range(Nsims):
        muestra = norm.rvs(loc=mu, scale=sigma, size=size)
        mu_estimado = sum(muestra)/size
        sigma_estimado = 0
        for i in muestra:
            sigma_estimado += (i - mu_estimado)**2
        sigma_estimado = math.sqrt(sigma_estimado/(size-1))
        d_j = estadistico_d(muestra, (lambda x: norm.cdf(x, mu_estimado, sigma_estimado)))
        if d_j > d0:
            pvalor += 1
    return pvalor/Nsims

print("Valor estimado de p-valor con normales:", pvalor_normal(mu_estimado, sigma_estimado, d0, len(M)))
print("En ambos casos p-valor > 0.05, con un nivel de rechazo del 5% no se rechaza H0")

Estadístico d: 0.19638944697995597
Valor estimado de p-valor con uniformes: 0.6719
Valor estimado de p-valor con normales: 0.226
En ambos casos p-valor > 0.05, con un nivel de rechazo del 5% no se rechaza H0
